In [2]:
# !pip install owlready2

     |████████████████████████████████| 23.8MB 7.5MB/s eta 0:00:01     |█████████████████████           | 15.6MB 7.5MB/s eta 0:00:02
    Running setup.py install for owlready2 ... done
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
from owlready2 import *

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [3]:
onto = get_ontology("file:///Users/kevinlin/Documents/classes/cs270/final-project/cs270-final-project/ontology/restaurant.owl").load()

In [4]:
list(onto.classes())

[restaurants.Ambience,
 restaurants.AmericanRestaurant,
 restaurants.Restaurant,
 restaurants.AsianFusionRestaurant,
 restaurants.AsianRestaurant,
 restaurants.Business,
 restaurants.CantoneseRestaurant,
 restaurants.ChineseRestaurant,
 restaurants.CasualAmbience,
 restaurants.EastAsianRestaurant,
 restaurants.Dimsum,
 restaurants.Specialization,
 restaurants.DiveyAmbience,
 restaurants.Dumplings,
 restaurants.TaiwaneseRestaurant,
 restaurants.HakkaRestaurant,
 restaurants.HipsterAmbience,
 restaurants.HongKongStyleCafe,
 restaurants.IntimateAmbience,
 restaurants.JapaneseRestaurant,
 restaurants.KoreanRestaurant,
 restaurants.PanAsianRestaurant,
 restaurants.RomanticAmbience,
 restaurants.ShanghaineseRestaurant,
 restaurants.SingaporeanRestaurant,
 restaurants.TouristyAmbience,
 restaurants.TrendyAmbience,
 restaurants.UpscaleAmbience,
 restaurants.Vegan,
 restaurants.Vegetarian]

In [ ]:
# TODO: create instances

# TODO: figure out how to create queries (i.e. given input, create necessary query to get results)

# TODO: develope specific algorithms / strategies for PSM (e.g. what happens when user wants to find more results or there are no results at all)